## Example Usage for the ndx-patterne-ogen-stimulation

In [1]:
import datetime
import numpy as np
from pynwb import NWBFile, NWBHDF5IO
from ndx_patterned_ogen import (
    SpatialLightModulator,
    LightSource,
    PatternedOptogeneticStimulusSite,
    PatternedOptogeneticStimulusTable,
    OptogeneticStimulusPattern,
    OptogeneticStimulusSite,
    OptogeneticStimulusTarget,
    SpiralScanning,
    TemporalFocusing,
)
from pynwb.ophys import PlaneSegmentation, ImageSegmentation, OpticalChannel

In [2]:
nwbfile = NWBFile(
    session_description="session_description",
    identifier="identifier",
    session_start_time=datetime.datetime.now(datetime.timezone.utc),
)

Define a `roi_table_region` in the imaging acquisition system, to link the holographic photostimulation to a set of ROIs

In [3]:
recording_device = nwbfile.create_device(
    name="2P_microscope",
    description="My two-photon microscope",
    manufacturer="The best microscope manufacturer",
)

optical_channel = OpticalChannel(
    name="OpticalChannel",
    description="an optical channel",
    emission_lambda=500.0,
)
imaging_plane = nwbfile.create_imaging_plane(
    name="ImagingPlane",
    optical_channel=optical_channel,
    imaging_rate=30.0,
    description="a very interesting part of the brain",
    device=recording_device,
    excitation_lambda=600.0,
    indicator="GFP",
    location="V1",
    grid_spacing=[0.01, 0.01],
    grid_spacing_unit="meters",
    origin_coords=[1.0, 2.0, 3.0],
    origin_coords_unit="meters",
)

n_rois = 10
plane_segmentation = PlaneSegmentation(
    name="PlaneSegmentation",
    description="output from segmenting my favorite imaging plane",
    imaging_plane=imaging_plane,
)
for _ in range(n_rois):
    plane_segmentation.add_roi(image_mask=np.zeros((10, 10)))

if nwbfile is not None:
    if "ophys" not in nwbfile.processing:
        nwbfile.create_processing_module("ophys", "ophys")
    nwbfile.processing["ophys"].add(plane_segmentation)

roi_table_region_1 = plane_segmentation.create_roi_table_region(region=list(np.arange(n_rois)), description="all ROIs")
roi_table_region_2 = plane_segmentation.create_roi_table_region(
    region=list(np.arange(3)), description="the first 3 ROIs"
)
roi_table_region_3 = plane_segmentation.create_roi_table_region(
    region=list(np.arange(n_rois - 4, n_rois)), description="the last 4 ROIs"
)

hologram_1 = OptogeneticStimulusTarget(name="Hologram1", stimulated_rois=roi_table_region_1)
nwbfile.add_lab_meta_data(hologram_1)
hologram_2 = OptogeneticStimulusTarget(name="Hologram2", stimulated_rois=roi_table_region_2)
nwbfile.add_lab_meta_data(hologram_2)
hologram_3 = OptogeneticStimulusTarget(name="Hologram3", stimulated_rois=roi_table_region_3)
nwbfile.add_lab_meta_data(hologram_3)

,image_mask
id,
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]"
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]"
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]"
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]"


Define optical devices that provide the stimulus

In [4]:
# metadata for spiatial light modulator
spatial_light_modulator = SpatialLightModulator(
    name="SpatialLightModulator",
    description="Generic description for the slm",
    model="slm model",
    manufacturer="slm manufacturer",
    size=[100, 100],
)
nwbfile.add_device(spatial_light_modulator)

# metadata for the light source
light_source = LightSource(
    name="Laser",
    model="laser model",
    manufacturer="laser manufacturer",
    stimulation_wavelength=600.0,  # nm
    description="Generic description for the laser",
    peak_power=70e-3,  # the peak power of stimulation in Watts
    intensity=0.005,  # the intensity of excitation in W/mm^2
    exposure_time=2.51e-13,  # the exposure time of the sample in seconds
    pulse_rate=1 / 2.51e-13,  # the pulse rate of the laser is in Hz
)
nwbfile.add_device(light_source)

# metadata for the stimulating device
stimulating_device = nwbfile.create_device(
    name="StimulatingDevice",
    description="Microsope used for optogenetic stimulation",
    manufacturer="device manufacturer",
)

Define stimulus patterns parameters

In [5]:
# metadata for a generic stimulus pattern
generic_pattern = OptogeneticStimulusPattern(
    name="OptogeneticStimulusPattern",
    description="beam pattern",
    sweep_size=5,  # um
    sweep_mask=np.zeros((5,5)),
)
nwbfile.add_lab_meta_data(generic_pattern)

# metadata for spiral scanning pattern
spiral_scanning = SpiralScanning(
    name="SpiralScanning",
    diameter=15,  # um
    height=10,  # um
    number_of_revolutions=5,
    description="scanning beam pattern",
)
nwbfile.add_lab_meta_data(spiral_scanning)

# metadata for temporal focusing pattern
temporal_focusing = TemporalFocusing(
    name="TemporalFocusing",
    description="scanless beam pattern",
    lateral_point_spread_function="9 um ± 0.7 um",
    axial_point_spread_function="32 um ± 1.6 um",
)
nwbfile.add_lab_meta_data(temporal_focusing)

TemporalFocusing ndx_patterned_ogen.patterned_ogen.TemporalFocusing at 0x139800105157600
Fields:
  axial_point_spread_function: 32 um ± 1.6 um
  description: scanless beam pattern
  lateral_point_spread_function: 9 um ± 0.7 um

Define stimulus method

In [6]:
# metadata for the stimulus methods
site = PatternedOptogeneticStimulusSite(
    name="PatternedOptogeneticStimulusSite",
    description="Scanning",  # Scanning or scanless method for shaping optogenetic light (e.g., diffraction limited points, 3D shot, disks, etc.).
    excitation_lambda=600.0,  # nm
    effector="ChR2",
    location="VISrl",
    device=stimulating_device,
    spatial_light_modulator=spatial_light_modulator,
    light_source=light_source,
)
nwbfile.add_ogen_site(site)

PatternedOptogeneticStimulusSite ndx_patterned_ogen.patterned_ogen.PatternedOptogeneticStimulusSite at 0x139800104041696
Fields:
  description: Scanning
  device: StimulatingDevice pynwb.device.Device at 0x139800103816080
Fields:
  description: Microsope used for optogenetic stimulation
  manufacturer: device manufacturer

  effector: ChR2
  excitation_lambda: 600.0
  light_source: Laser ndx_patterned_ogen.patterned_ogen.LightSource at 0x139800104270736
Fields:
  description: Generic description for the laser
  exposure_time: 2.51e-13
  intensity: 0.005
  manufacturer: laser manufacturer
  model: laser model
  peak_power: 0.07
  pulse_rate: 3984063745019.9204
  stimulation_wavelength: 600.0

  location: VISrl
  spatial_light_modulator: SpatialLightModulator ndx_patterned_ogen.patterned_ogen.SpatialLightModulator at 0x139800104040256
Fields:
  description: Generic description for the slm
  manufacturer: slm manufacturer
  model: slm model
  size: [100 100]

Define the stimulus sequences on the holograms previously defined in the imaging frame coordinates 

In [7]:
stimulus_table = PatternedOptogeneticStimulusTable(
    name="PatternedOptogeneticStimulusTable", description="Patterned stimulus"
)
stimulus_table.add_interval(
    start_time=0.0,
    stop_time=1.0,
    power=70e-3,
    frequency=20.0,
    pulse_width=0.1,
    stimulus_pattern=temporal_focusing,
    targets=hologram_1,
    stimulus_site=site,
)
stimulus_table.add_interval(
    start_time=0.5,
    stop_time=1.0,
    power=50e-3,
    frequency=20.0,
    pulse_width=0.1,
    stimulus_pattern=spiral_scanning,
    targets=hologram_2,
    stimulus_site=site,
)
stimulus_table.add_interval(
    start_time=0.8,
    stop_time=1.7,
    power=40e-3,
    frequency=20.0,
    pulse_width=0.1,
    stimulus_pattern=generic_pattern,
    targets=hologram_3,
    stimulus_site=site,
)
nwbfile.add_time_intervals(stimulus_table)

,start_time,stop_time,power,targets,stimulus_pattern,stimulus_site,frequency,pulse_width
id,,,,,,,,
0,0.0,1.0,0.07,Hologram1 ndx_patterned_ogen.patterned_ogen.OptogeneticStimulusTarget at 0x139801493387120\nFields:\n stimulated_rois: rois <class 'hdmf.common.table.DynamicTableRegion'>\n,TemporalFocusing ndx_patterned_ogen.patterned_ogen.TemporalFocusing at 0x139800105157600\nFields:\n axial_point_spread_function: 32 um ± 1.6 um\n description: scanless beam pattern\n lateral_point_spread_function: 9 um ± 0.7 um\n,PatternedOptogeneticStimulusSite ndx_patterned_ogen.patterned_ogen.PatternedOptogeneticStimulusSite at 0x139800104041696\nFields:\n description: Scanning\n device: StimulatingDevice pynwb.device.Device at 0x139800103816080\nFields:\n description: Microsope used for optogenetic stimulation\n manufacturer: device manufacturer\n\n effector: ChR2\n excitation_lambda: 600.0\n light_source: Laser ndx_patterned_ogen.patterned_ogen.LightSource at 0x139800104270736\nFields:\n description: Generic description for the laser\n exposure_time: 2.51e-13\n intensity: 0.005\n manufacturer: laser manufacturer\n model: laser model\n peak_power: 0.07\n pulse_rate: 3984063745019.9204\n stimulation_wavelength: 600.0\n\n location: VISrl\n spatial_light_modulator: SpatialLightModulator ndx_patterned_ogen.patterned_ogen.SpatialLightModulator at 0x139800104040256\nFields:\n description: Generic description for the slm\n manufacturer: slm manufacturer\n model: slm model\n size: [100 100]\n\n,20.0,0.1
1,0.5,1.0,0.05,Hologram2 ndx_patterned_ogen.patterned_ogen.OptogeneticStimulusTarget at 0x139800104034352\nFields:\n stimulated_rois: rois <class 'hdmf.common.table.DynamicTableRegion'>\n,SpiralScanning ndx_patterned_ogen.patterned_ogen.SpiralScanning at 0x139800126610256\nFields:\n description: scanning beam pattern\n diameter: 15\n height: 10\n number_of_revolutions: 5\n,PatternedOptogeneticStimulusSite ndx_patterned_ogen.patterned_ogen.PatternedOptogeneticStimulusSite at 0x139800104041696\nFields:\n description: Scanning\n device: StimulatingDevice pynwb.device.Device at 0x139800103816080\nFields:\n description: Microsope used for optogenetic stimulation\n manufacturer: device manufacturer\n\n effector: ChR2\n excitation_lambda: 600.0\n light_source: Laser ndx_patterned_ogen.patterned_ogen.LightSource at 0x139800104270736\nFields:\n description: Generic description for the laser\n exposure_time: 2.51e-13\n intensity: 0.005\n manufacturer: laser manufacturer\n model: laser model\n peak_power: 0.07\n pulse_rate: 3984063745019.9204\n stimulation_wavelength: 600.0\n\n location: VISrl\n spatial_light_modulator: SpatialLightModulator ndx_patterned_ogen.patterned_ogen.SpatialLightModulator at 0x139800104040256\nFields:\n description: Generic description for the slm\n manufacturer: slm manufacturer\n model: slm model\n size: [100 100]\n\n,20.0,0.1
2,0.8,1.7,0.04,Hologram3 ndx_patterned_ogen.patterned_ogen.OptogeneticStimulusTarget at 0x139800104034592\nFields:\n stimulated_rois: rois <class 'hdmf.common.table.DynamicTableRegion'>\n,OptogeneticStimulusPattern ndx_patterned_ogen.patterned_ogen.OptogeneticStimulusPattern at 0x139800104041456\nFields:\n description: beam pattern\n sweep_mask: [[0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0.]\n [0. 0. 0. 0. 0.]]\n sweep_size: 5\n,PatternedOptogeneticStimulusSite ndx_patterned_ogen.patterned_ogen.PatternedOptogeneticStimulusSite at 0x139800104041696\nFields:\n description: Scanning\n device: StimulatingDevice pynwb.device.Device at 0x139800103816080\nFields:\n description: Microsope used for optogenetic stimulation\n manufacturer: device manufacturer\n\n effector: ChR2\n excitation_lambda: 600.0\n light_source: Laser ndx_patterned_ogen.patterned_ogen.LightSource at 0x139800104270736\nFields:\n description: Generic description for the laser\n exposure_time: 2.51e-13\n intensity: 0.005\n manufacturer: laser manufacturer\n model: laser model\n peak_power: 0.07\n pulse_rate: 3984063745019.

Write and read the NWB File

In [8]:
nwbfile_path = "basics_tutorial_patterned_ogen.nwb"
with NWBHDF5IO(nwbfile_path, mode="w") as io:
    io.write(nwbfile)

with NWBHDF5IO(nwbfile_path, mode="r") as io:
    nwbfile_in = io.read()

/home/amtra/miniconda3/envs/ndx_patterned_ogen_env/lib/python3.12/site-packages/hdmf/build/objectmapper.py:671: MissingRequiredBuildWarning: OptogeneticStimulusTarget 'Hologram1' is missing required value for attribute 'additional_targeted_rois'.
  warnings.warn(msg, MissingRequiredBuildWarning)
/home/amtra/miniconda3/envs/ndx_patterned_ogen_env/lib/python3.12/site-packages/hdmf/build/objectmapper.py:671: MissingRequiredBuildWarning: OptogeneticStimulusTarget 'Hologram2' is missing required value for attribute 'additional_targeted_rois'.
  warnings.warn(msg, MissingRequiredBuildWarning)
/home/amtra/miniconda3/envs/ndx_patterned_ogen_env/lib/python3.12/site-packages/hdmf/build/objectmapper.py:671: MissingRequiredBuildWarning: OptogeneticStimulusTarget 'Hologram3' is missing required value for attribute 'additional_targeted_rois'.
  warnings.warn(msg, MissingRequiredBuildWarning)
/home/amtra/miniconda3/envs/ndx_patterned_ogen_env/lib/python3.12/site-packages/hdmf/build/objectmapper.py:67

ConstructError: (root/general/Hologram1 GroupBuilder {'attributes': {'namespace': 'ndx-patterned-ogen', 'neurodata_type': 'OptogeneticStimulusTarget', 'object_id': '75cebb1d-d0ce-47a2-bddc-331f6ceee2fe'}, 'groups': {}, 'datasets': {'rois': root/general/Hologram1/rois DatasetBuilder {'attributes': {'description': 'all ROIs', 'namespace': 'hdmf-common', 'neurodata_type': 'DynamicTableRegion', 'object_id': '8972d00d-14d9-4ad8-8db9-c962e31841a2', 'table': root/processing/ophys/PlaneSegmentation GroupBuilder {'attributes': {'colnames': array(['image_mask'], dtype=object), 'description': 'output from segmenting my favorite imaging plane', 'namespace': 'core', 'neurodata_type': 'PlaneSegmentation', 'object_id': '015618cd-3550-4658-8cb7-06da7424d496'}, 'groups': {'reference_images': root/processing/ophys/PlaneSegmentation/reference_images GroupBuilder {'attributes': {}, 'groups': {}, 'datasets': {}, 'links': {}}}, 'datasets': {'id': root/processing/ophys/PlaneSegmentation/id DatasetBuilder {'a